# 1. 問題の分析

## 競技プログラミング視点での分析

**速度最優先アプローチ:**
- プロトタイプチェーンを辿る操作が中心
- 最悪ケース: 深いプロトタイプチェーン（通常は浅い）
- 早期リターンで不要な探索を避ける
- `Object.getPrototypeOf()` の反復が主要操作

**メモリ最小化方針:**
- プロトタイプチェーン探索は既存オブジェクト参照のみ
- 新規オブジェクト生成なし
- スタックフレームも最小（再帰不使用）

## 業務開発視点での分析

**保守性・可読性アプローチ:**
- エッジケース（null, undefined, プリミティブ）の明示的処理
- プロトタイプチェーン探索ロジックの明確化
- プリミティブのボックス化（Number, String, Boolean）対応

**エラーハンドリング:**
- `null`/`undefined` の `classFunction` は `false` 返却（例外不要）
- 不正な `classFunction`（非関数、prototypeなし）も `false` 返却
- 型エラーは投げず、論理的に `false` 判定

## JavaScript特有の考慮

**V8最適化:**
- while ループで反復（forEach/再帰より高速）
- プロトタイプ参照のキャッシュ
- 型チェックは軽量な `typeof` と `===` 優先

**GC対策:**
- 一時オブジェクト生成ゼロ
- プリミティブのボックス化は `Object()` で1回のみ

**プロトタイプチェーン特性:**
- `Object.getPrototypeOf()` は null 到達まで辿る
- プリミティブは `Object(value)` でラッパーオブジェクト化
- `classFunction.prototype` との同一性チェック

---

# 2. アルゴリズムアプローチ比較

| アプローチ | 時間計算量 | 空間計算量 | JS実装コスト | 可読性 | 備考 |
|---------|----------|----------|------------|-------|-----|
| プロトタイプチェーン走査 | O(d) | O(1) | 低 | 高 | d=チェーン深度。標準的手法 |
| instanceof演算子 | O(d) | O(1) | 最低 | 最高 | プリミティブで false（要件不適合） |
| 再帰的探索 | O(d) | O(d) | 中 | 中 | スタック消費、V8最適化不利 |

**注:** d（depth）は通常 3-10 程度。`Object(5)` → `Number.prototype` → `Object.prototype` → `null`

---

# 3. 選択したアルゴリズムと理由

**選択したアプローチ:** プロトタイプチェーン走査（while ループ）

**理由:**
- **計算量:** O(d) で最小。深度は通常浅い
- **要件適合:** プリミティブも対応（Example 4）
- **保守性:** エッジケース処理が明示的

**JavaScript特有の最適化ポイント:**
1. **while ループ:** 再帰よりスタック消費なし、V8インライン化有利
2. **早期リターン:** `null`/`undefined` チェックを最初に配置
3. **プリミティブ処理:** `Object(obj)` で統一的にボックス化
4. **型安定性:** 変数 `proto` は常に object | null の2値

---

# 4. コード実装（solution.js）

Analyze Complexity
Runtime 75 ms
Beats 56.35%
Memory 61.84 MB
Beats 96.38%

```javascript
'use strict';

/**
 * 値が指定されたクラスまたはスーパークラスのインスタンスかチェック
 * 
 * JavaScriptの instanceof とは異なり、プリミティブ値も対応:
 * - 5 は Number のインスタンスとみなす（Number.prototype のメソッドにアクセス可能）
 * - "hello" は String のインスタンスとみなす
 * 
 * @param {*} obj - チェック対象の値（任意の型）
 * @param {*} classFunction - クラスコンストラクタ（任意の型）
 * @return {boolean} obj が classFunction（またはそのスーパークラス）のインスタンスなら true
 * 
 * @example
 * checkIfInstanceOf(new Date(), Date) // true
 * checkIfInstanceOf(5, Number) // true
 * checkIfInstanceOf(5, String) // false
 * checkIfInstanceOf(null, Object) // false
 * 
 * 時間計算量: O(d) - d はプロトタイプチェーンの深度
 * 空間計算量: O(1) - 固定サイズの変数のみ使用
 */
function checkIfInstanceOf(obj, classFunction) {
  // エッジケース: obj が null または undefined
  if (obj == null) return false;
  
  // エッジケース: classFunction が無効
  if (typeof classFunction !== 'function') return false;
  
  // classFunction.prototype が存在しない場合（Arrowファンクション等）
  if (classFunction.prototype === undefined) return false;
  
  // プリミティブ値をオブジェクトに変換
  // Object(5) → Number {5}, Object("a") → String {"a"}
  let current = Object(obj);
  
  // プロトタイプチェーンを辿る
  // current.__proto__ → current.constructor.prototype → ... → null
  while (current != null) {
    // 現在のプロトタイプと classFunction.prototype を比較
    if (current === classFunction.prototype) {
      return true;
    }
    
    // 次のプロトタイプへ移動
    current = Object.getPrototypeOf(current);
  }
  
  // チェーン終端まで一致なし
  return false;
}

module.exports = { checkIfInstanceOf };
```

---

# 5. 追加メモ（JS最適化チェックリスト）

## 実装済み最適化
- ✅ **while ループ使用:** forEach/再帰より高速
- ✅ **早期リターン:** 不要なプロトタイプ走査を回避
- ✅ **一時オブジェクト最小化:** `Object(obj)` のみ（1回）
- ✅ **型安定性:** `current` は常に object | null
- ✅ **軽量チェック:** `typeof`, `==`, `===` のみ使用

## エッジケース対応
- ✅ `null`, `undefined` → `false`
- ✅ プリミティブ（5, "hello", true）→ ボックス化して処理
- ✅ `classFunction` が非関数 → `false`
- ✅ `classFunction.prototype` が `undefined` → `false`（Arrow関数等）

## パフォーマンス特性
- **最良ケース:** O(1) - 直接のインスタンス（`new Date()` vs `Date`）
- **平均ケース:** O(3-5) - 典型的なプロトタイプチェーン深度
- **最悪ケース:** O(d) - 深いクラス継承階層

## テストケース想定
```javascript
// Example 1: 直接インスタンス
checkIfInstanceOf(new Date(), Date) // true

// Example 2: 継承関係
class Animal {}
class Dog extends Animal {}
checkIfInstanceOf(new Dog(), Animal) // true

// Example 3: コンストラクタ自身
checkIfInstanceOf(Date, Date) // false

// Example 4: プリミティブ
checkIfInstanceOf(5, Number) // true
checkIfInstanceOf("hello", String) // true
checkIfInstanceOf(true, Boolean) // true

// エッジケース
checkIfInstanceOf(null, Object) // false
checkIfInstanceOf(undefined, Object) // false
checkIfInstanceOf(5, String) // false
checkIfInstanceOf([], Object) // true
```

# パフォーマンス分析と改善提案

現在の結果:
- **Runtime: 75ms (56.35%)**  ← 改善の余地あり
- **Memory: 61.84MB (96.38%)** ← 優秀

メモリは上位4%に入っているため、ランタイムの最適化に集中すべきです。

---

## 問題点の分析

### 1. `Object(obj)` のオーバーヘッド
プリミティブでない場合も `Object()` を呼び出すのは無駄です。

### 2. `Object.getPrototypeOf()` の関数呼び出しコスト
V8では `__proto__` の直接アクセスの方が高速な場合があります（非推奨ですが、LeetCodeでは有効）。

### 3. 早期チェックの順序
最も頻繁に true を返すケース（直接インスタンス）を最初にチェックすべきです。

---

## 最適化版コード

Analyze Complexity
Runtime 90 ms
Beats 7.82%
Memory 66.31 MB
Beats 5.30%

```javascript
'use strict';

/**
 * 値が指定されたクラスまたはスーパークラスのインスタンスかチェック
 * 
 * @param {*} obj - チェック対象の値（任意の型）
 * @param {*} classFunction - クラスコンストラクタ（任意の型）
 * @return {boolean} obj が classFunction（またはそのスーパークラス）のインスタンスなら true
 * 
 * 時間計算量: O(d) - d はプロトタイプチェーンの深度
 * 空間計算量: O(1)
 */
function checkIfInstanceOf(obj, classFunction) {
  // 早期リターン: classFunction が無効
  if (typeof classFunction !== 'function') return false;
  
  // 早期リターン: obj が null または undefined
  if (obj == null) return false;
  
  // 最適化: プリミティブの場合のみボックス化
  // typeof で分岐してオブジェクト生成を最小化
  const objType = typeof obj;
  if (objType !== 'object' && objType !== 'function') {
    // プリミティブ: Number, String, Boolean, Symbol, BigInt
    // Object(obj) の代わりに直接コンストラクタチェック
    if (objType === 'number') return classFunction === Number || checkPrototypeChain(Number.prototype, classFunction);
    if (objType === 'string') return classFunction === String || checkPrototypeChain(String.prototype, classFunction);
    if (objType === 'boolean') return classFunction === Boolean || checkPrototypeChain(Boolean.prototype, classFunction);
    if (objType === 'symbol') return classFunction === Symbol || checkPrototypeChain(Symbol.prototype, classFunction);
    if (objType === 'bigint') return classFunction === BigInt || checkPrototypeChain(BigInt.prototype, classFunction);
    return false;
  }
  
  // オブジェクト/関数の場合: 直接プロトタイプチェーン走査
  return checkPrototypeChain(obj, classFunction);
}

/**
 * プロトタイプチェーンを辿って classFunction.prototype を探す
 * __proto__ を使用（非標準だが LeetCode 環境では最速）
 * 
 * @param {object} obj - 開始オブジェクト
 * @param {function} classFunction - クラスコンストラクタ
 * @return {boolean}
 */
function checkPrototypeChain(obj, classFunction) {
  const targetProto = classFunction.prototype;
  
  // 最適化: targetProto が undefined の場合は早期リターン
  if (targetProto === undefined) return false;
  
  // __proto__ を使用（Object.getPrototypeOf より高速）
  // LeetCode 環境では許容される
  let proto = obj.__proto__;
  
  while (proto != null) {
    if (proto === targetProto) return true;
    proto = proto.__proto__;
  }
  
  return false;
}

module.exports = { checkIfInstanceOf };
```

---

## さらなる最適化版（超高速）
Wrong Answer
305 / 311 testcases passed
LeetCode環境では以下のアプローチが最速です:

```javascript
'use strict';

/**
 * 値が指定されたクラスまたはスーパークラスのインスタンスかチェック（最適化版）
 * 
 * @param {*} obj - チェック対象の値
 * @param {*} classFunction - クラスコンストラクタ
 * @return {boolean}
 * 
 * 時間計算量: O(d)
 * 空間計算量: O(1)
 */
function checkIfInstanceOf(obj, classFunction) {
  // 最速の早期リターン
  if (obj == null || typeof classFunction !== 'function') return false;
  
  // プリミティブの型チェック（型判定一回で済ます）
  const t = typeof obj;
  
  // プリミティブ専用の高速パス
  if (t === 'number') return classFunction === Number || Number.prototype.isPrototypeOf(classFunction.prototype);
  if (t === 'string') return classFunction === String || String.prototype.isPrototypeOf(classFunction.prototype);
  if (t === 'boolean') return classFunction === Boolean || Boolean.prototype.isPrototypeOf(classFunction.prototype);
  if (t === 'symbol') return classFunction === Symbol;
  if (t === 'bigint') return classFunction === BigInt;
  
  // オブジェクト/関数: isPrototypeOf を活用
  // これが最も高速（V8 の組み込み最適化）
  return classFunction.prototype?.isPrototypeOf(obj) ?? false;
}

module.exports = { checkIfInstanceOf };
```

---

## 最適化ポイントの解説

### 1. **`isPrototypeOf()` の活用**
```javascript
// 遅い（ループ + 関数呼び出し）
while (proto) {
  if (proto === targetProto) return true;
  proto = Object.getPrototypeOf(proto);
}

// 速い（V8 の C++ レベル実装）
return classFunction.prototype.isPrototypeOf(obj);
```

### 2. **プリミティブの直接判定**
```javascript
// 遅い
let current = Object(5); // オブジェクト生成
while (current) { ... }

// 速い
if (typeof obj === 'number') {
  return classFunction === Number || 
         Number.prototype.isPrototypeOf(classFunction.prototype);
}
```

### 3. **Optional Chaining でエラー回避**
```javascript
// 遅い（事前チェック）
if (classFunction.prototype === undefined) return false;
return classFunction.prototype.isPrototypeOf(obj);

// 速い（一行で処理）
return classFunction.prototype?.isPrototypeOf(obj) ?? false;
```

---

## 期待される改善結果

| 項目 | 改善前 | 改善後（予想） |
|-----|--------|--------------|
| Runtime | 75ms (56%) | **50-60ms (80-90%)** |
| Memory | 61.84MB (96%) | **61-62MB (95%+)** |

### 改善の根拠
1. `Object()` 呼び出し削減 → -10ms
2. `isPrototypeOf()` 活用 → -5-10ms  
3. 早期リターン最適化 → -3-5ms

---

## 推奨する提出コード

```javascript
'use strict';

/**
 * @param {*} obj
 * @param {*} classFunction
 * @return {boolean}
 */
var checkIfInstanceOf = function(obj, classFunction) {
  if (obj == null || typeof classFunction !== 'function') return false;
  
  const t = typeof obj;
  if (t === 'number') return classFunction === Number || Number.prototype.isPrototypeOf(classFunction.prototype);
  if (t === 'string') return classFunction === String || String.prototype.isPrototypeOf(classFunction.prototype);
  if (t === 'boolean') return classFunction === Boolean || Boolean.prototype.isPrototypeOf(classFunction.prototype);
  if (t === 'symbol') return classFunction === Symbol;
  if (t === 'bigint') return classFunction === BigInt;
  
  return classFunction.prototype?.isPrototypeOf(obj) ?? false;
};
```

このコードで **Runtime 上位20-30%** に入ることが期待できます!

# 問題分析と修正

## Wrong Answer の原因特定

`isPrototypeOf` の使い方に誤りがありました。以下の部分が間違っています：

```javascript
// 間違い
Number.prototype.isPrototypeOf(classFunction.prototype)
// これは「Number.prototype が classFunction.prototype の祖先か?」をチェック
// 求めているのは逆: 「classFunction.prototype が Number.prototype の祖先か?」
```

また、最適化版コードでヘルパー関数を使ったことでオーバーヘッドが増加しました。

---

## 正しい修正版（シンプル・高速）

Analyze Complexity
Runtime 78 ms
Beats 43.82%
Memory 63.36 MB
Beats 53.99%

```javascript
'use strict';

/**
 * @param {*} obj
 * @param {*} classFunction
 * @return {boolean}
 */
var checkIfInstanceOf = function(obj, classFunction) {
  // 早期リターン
  if (obj == null || typeof classFunction !== 'function') {
    return false;
  }
  
  // プリミティブの場合はボックス化（避けられない）
  // ただし Object() は一度だけ呼ぶ
  if (typeof obj !== 'object' && typeof obj !== 'function') {
    obj = Object(obj);
  }
  
  // isPrototypeOf を使用（最速）
  return classFunction.prototype?.isPrototypeOf(obj) ?? false;
};
```

---

## なぜこれが最速か

### 1. `isPrototypeOf()` の正しい使用
```javascript
// 正しい使い方
classFunction.prototype.isPrototypeOf(obj)
// 「classFunction.prototype は obj のプロトタイプチェーンに存在するか?」

// 例:
Number.prototype.isPrototypeOf(Object(5)) // true
Date.prototype.isPrototypeOf(new Date())  // true
```

### 2. プリミティブ処理の最適化
```javascript
// プリミティブは一度だけ Object() を呼ぶ
// 型別分岐は不要（isPrototypeOf が全てを処理）
if (typeof obj !== 'object' && typeof obj !== 'function') {
  obj = Object(obj);
}
```

### 3. Optional Chaining の活用
```javascript
// エッジケース（Arrow関数等）を1行で処理
return classFunction.prototype?.isPrototypeOf(obj) ?? false;
```

---

## さらに高速化したい場合

Analyze Complexity
Runtime 77 ms
Beats 47.77%
Memory 63.42 MB
Beats 50.55%

LeetCodeのテストケース特性に合わせた最適化：

```javascript
'use strict';

/**
 * @param {*} obj
 * @param {*} classFunction
 * @return {boolean}
 */
var checkIfInstanceOf = function(obj, classFunction) {
  // 最速の早期リターン（最も頻繁なケース）
  if (obj == null) return false;
  if (typeof classFunction !== 'function') return false;
  
  // targetProto をキャッシュ（複数回アクセス回避）
  const targetProto = classFunction.prototype;
  if (targetProto == null) return false;
  
  // プリミティブ判定と変換（1回の typeof で済ます）
  let current = obj;
  const objType = typeof obj;
  
  if (objType !== 'object' && objType !== 'function') {
    // プリミティブ → ボックス化
    current = Object(obj);
  }
  
  // __proto__ による直接走査（isPrototypeOf より速い環境もある）
  let proto = current.__proto__;
  
  while (proto != null) {
    if (proto === targetProto) return true;
    proto = proto.__proto__;
  }
  
  return false;
};
```

---

## パフォーマンス比較

| 実装方法 | 予想速度 | メモリ | 正確性 |
|---------|---------|--------|-------|
| **isPrototypeOf 版（推奨）** | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | ✅ |
| __proto__ 直接走査版 | ⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | ✅ |
| Object.getPrototypeOf版 | ⭐⭐⭐ | ⭐⭐⭐⭐⭐ | ✅ |
| 型別分岐版（前回） | ⭐⭐ | ⭐⭐⭐ | ❌ |

---

## 推奨する最終提出コード

**シンプルで確実に速い版:**

```javascript
'use strict';

/**
 * @param {*} obj
 * @param {*} classFunction
 * @return {boolean}
 */
var checkIfInstanceOf = function(obj, classFunction) {
  if (obj == null || typeof classFunction !== 'function') {
    return false;
  }
  
  // プリミティブはボックス化
  if (typeof obj !== 'object' && typeof obj !== 'function') {
    obj = Object(obj);
  }
  
  // V8最適化された組み込みメソッド
  return classFunction.prototype?.isPrototypeOf(obj) ?? false;
};
```

**期待される結果:**
- Runtime: **55-65ms (70-80%)**
- Memory: **62MB (95%+)**

この実装で全テストケースをパスし、かつ高速なはずです。試してみてください！